In [4]:
from datasets import load_dataset, DownloadConfig

download_config = DownloadConfig(delete_extracted=True)
dataset = load_dataset("./lua_sources", split="train", download_config=download_config)

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
import os
import psutil


print(f"Number of python files code in dataset : {len(dataset)}")
ds_size = sum(os.stat(f["filename"]).st_size for f in dataset.cache_files)
print(f"Dataset size (cache file): {ds_size / 2**30:.2f}GB")
print(f"RAM used: {psutil.Process(os.getpid()).memory_info().rss >> 20}MB")

Number of python files code in dataset : 1339692
Dataset size (cache file): 7.29GB
RAM used: 610MB


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

loading configuration file config.json from cache at /Users/RikitoNoto/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "trans

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

loading file vocab.json from cache at /Users/RikitoNoto/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/vocab.json
loading file merges.txt from cache at /Users/RikitoNoto/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/merges.txt
loading file tokenizer.json from cache at /Users/RikitoNoto/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /Users/RikitoNoto/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenizer_config.json
loading configuration file config.json from cache at /Users/RikitoNoto/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": 

In [7]:
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode

byte_to_unicode_map = bytes_to_unicode()
unicode_to_byte_map = dict((v, k) for k, v in byte_to_unicode_map.items())
base_vocab = list(unicode_to_byte_map.keys())

In [23]:
from tqdm import tqdm
length = 100000
dataset_name = "rikitonoto/lua_copilot_train"
dataset = load_dataset(dataset_name, split="train", streaming=True)
iter_dataset = iter(dataset)

def batch_iterator(batch_size: int = 10):
  for _ in tqdm(range(0,length, batch_size)):
    yield [next(iter_dataset)["content"] for _ in range(batch_size)]

new_tokenizer = tokenizer.train_new_from_iterator(
  batch_iterator(),
  vocab_size=8192,
  initial_alphabet=base_vocab,
)


100%|██████████| 10000/10000 [00:40<00:00, 248.42it/s]


In [24]:
tokens = sorted(new_tokenizer.vocab.items(), key=lambda x: x[1], reverse=False)
print([f"{tokenizer.convert_tokens_to_string([t])}" for t, _ in tokens[257:280]])
print([f"{tokenizer.convert_tokens_to_string([t])}" for t, _ in tokens[-20:]])


['  ', '--', '    ', '----', ' =', 'en', 'er', 'on', ' t', 'in', 're', '   ', 'al', '--------', 'at', '\t\t', 'se', '\n    ', 'lo', 'st', 'ion', 'end', ' "']
['اف', 'LIST', ' tooltip', 'х', 'Border', '"\',', ' `"..', 'pot', ' txt', '자', 'ikipedia', '�', '\n   \n  ', '304', 'CHECK', 'UG', 'Mount', 'ae', 'ko', ' ipv']


In [25]:
keywords = ["and","break","do","else","elseif","end","false","for","function","if","in","local","nil","not","or","repeat","return","then","true","until","while","#","%","(",")","*","+",",","-",".","..","...","/",":",";","<","<=","=","==",">",">=","[","]","^","{","}","~="]
for keyw in keywords:
  if keyw not in new_tokenizer.vocab:
    print(f"No, keyword `{keyw}` is not in the vocabulary.")

No, keyword `until` is not in the vocabulary.


In [26]:
tes = new_tokenizer("""
function Direction.new(direction)
  return {
    direction = direction,
  }
end
""").tokens()
print([f"{tokenizer.convert_tokens_to_string([t])}" for t in tes])

['\n', 'function', ' D', 'irection', '.', 'new', '(', 'direction', ')', '\n ', ' return', ' {', '\n   ', ' direction', ' =', ' direction', ',', '\n ', ' }', '\n', 'end', '\n']


In [27]:
# トークナイザーの保存
new_tokenizer.push_to_hub("lua_tokenizer")

tokenizer config file saved in /var/folders/44/05p4sg5d51766385p76sfylm0000gn/T/tmpey91gigt/tokenizer_config.json
Special tokens file saved in /var/folders/44/05p4sg5d51766385p76sfylm0000gn/T/tmpey91gigt/special_tokens_map.json
Uploading the following files to rikitonoto/lua_tokenizer: tokenizer_config.json,special_tokens_map.json,tokenizer.json,README.md,merges.txt,vocab.json


CommitInfo(commit_url='https://huggingface.co/rikitonoto/lua_tokenizer/commit/7fd0baefe2b34b607dc528bde1816afb0df2e2e6', commit_message='Upload tokenizer', commit_description='', oid='7fd0baefe2b34b607dc528bde1816afb0df2e2e6', pr_url=None, pr_revision=None, pr_num=None)